Start by Installing Tensorflow 2.8.0

In [10]:
!pip install protobuf==3.20.1
!pip install --default-timeout=100 tensorflow==2.8.0
import tensorflow    # it is used to contruct neural networks
print(tensorflow.__version__)


2.8.0


After restarting the runtime, verify that your version is 2.8.0

In [11]:
import tensorflow
print(tensorflow.__version__)

2.8.0


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Installations of visvis and gaze360

In [13]:
!pip install --default-timeout=100 visvis

In [14]:

!cp /content/drive/MyDrive/PW_2024/getheadbb.py .
!cp /content/drive/MyDrive/PW_2024/modelo.py .
! git clone https://github.com/erkil1452/gaze360
! wget http://gaze360.csail.mit.edu/files/gaze360_model.pth.tar

fatal: destination path 'gaze360' already exists and is not an empty directory.
--2024-12-01 11:18:09--  http://gaze360.csail.mit.edu/files/gaze360_model.pth.tar
Resolving gaze360.csail.mit.edu (gaze360.csail.mit.edu)... 128.52.131.233
Connecting to gaze360.csail.mit.edu (gaze360.csail.mit.edu)|128.52.131.233|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gaze360.csail.mit.edu/files/gaze360_model.pth.tar [following]
--2024-12-01 11:18:09--  https://gaze360.csail.mit.edu/files/gaze360_model.pth.tar
Connecting to gaze360.csail.mit.edu (gaze360.csail.mit.edu)|128.52.131.233|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58370849 (56M) [application/x-tar]
Saving to: ‘gaze360_model.pth.tar.1’

gaze360_model.pth.t 100%[===================>]  55.67M  22.7MB/s    in 2.5s    

2024-12-01 11:18:12 (22.7 MB/s) - ‘gaze360_model.pth.tar.1’ saved [58370849/58370849]



Imports of the packages required

In [15]:
import random
from PIL import Image
import math
import numpy as np
from getheadbb import HeadBB
from modelo import Modelo
import os
import imageio
import time
import datetime

import sys
import torch
import torchvision.transforms as transforms
import argparse
import cv2
import json

Functions that will be used in the main program

In [16]:
def tensor2np(x,y):
    outputx = torch.zeros(x.size(0),2)
    outputy = torch.zeros(y.size(0),2)
    outputx[:,0] = x[:,0]
    outputx[:,1] = x[:,1]
    outputy[:,0] = y[:,0]
    outputy[:,1] = y[:,1]

    confidencef = outputx.detach().numpy()
    confidencef = confidencef.reshape((-1))

    output_gazef = outputy.detach().numpy()
    output_gazef = output_gazef.reshape((-1))

    return confidencef, output_gazef


def spherical2cartesial(x):
    output = torch.zeros(x.size(0),3)
    output[:,2] = -torch.cos(x[:,1])*torch.cos(x[:,0])
    output[:,0] = torch.cos(x[:,1])*torch.sin(x[:,0])
    output[:,1] = torch.sin(x[:,1])
    return output

def find_id(bbox,id_dict):
    id_final = None
    max_iou = 0.1
    for k in id_dict.keys():
        if(compute_iou(bbox,id_dict[k][0])>max_iou):
            id_final = k
            max_iou = compute_iou(bbox,id_dict[k][0])
    return id_final

def compute_iou(bb1,bb2):
    # determine the coordinates of the intersection rectangle
    x_left = max(bb1[0], bb2[0])
    y_top = max(bb1[1], bb2[1])
    x_right = min(bb1[2], bb2[2])
    y_bottom = min(bb1[3], bb2[3])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1[2]-bb1[0]) * (bb1[3]-bb1[1])
    bb2_area = (bb2[2]-bb2[0]) * (bb2[3]-bb2[1])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    eps = 1e-8

    if iou <= 0.0  or iou > 1.0 + eps: return 0.0

    return iou

Uploading the models of Gaze360 and Yolo

In [17]:

sys.path.append('/content/drive/MyDrive/PW_2024/HeadPoseEstimation-WHENet/') #this has to be changed according to our model
sys.path.append('/content/gaze360/code/')
from model import GazeLSTM
model = GazeLSTM() #name of the model; short memory. RGB images are put inside.
model = torch.nn.DataParallel(model).cuda()
model.cuda()
checkpoint = torch.load('/content/gaze360_model.pth.tar')
model.load_state_dict(checkpoint['state_dict'])
model.eval()
print(sys.path)
from yolo_v3.yolo_postprocess import YOLO

<ipython-input-17-61335c07c817>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('/content/gaze360_model.pth.tar')


['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/usr/local/lib/python3.10/dist-packages/setuptools/_vendor', '/root/.ipython', '/tmp/tmpvz2s84_l', '/content/drive/MyDrive/PW_2024/HeadPoseEstimation-WHENet/', '/content/gaze360/code/', '/content/drive/MyDrive/PW_2024/HeadPoseEstimation-WHENet/', '/content/gaze360/code/']


Pass the arguments required by Yolo

In [18]:
parser = argparse.ArgumentParser(description='whenet demo with yolo')
parser.add_argument('--display', type=str, default='simple', help='display all euler angle (simple, full)')
parser.add_argument('--score', type=float, default=0.3, help='yolo confidence score threshold')
parser.add_argument('--iou', type=float, default=0.3, help='yolo iou threshold')
parser.add_argument('--gpu', type=str, default='0', help='gpu')
parser.add_argument('--output', type=str, default='test.avi', help='output video name')
args = parser.parse_args(args=[])

# Yolo
yolo = YOLO(**vars(args))
# Loading pre-trained model
image_normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])



Instructions for updating:
Colocations handled automatically by placer.


/content/drive/MyDrive/PW_2024/HeadPoseEstimation-WHENet/yolo_v3/data/head_detect.h5 model, anchors, and classes loaded.


In [20]:

## Define model
writing_mode = 1
## Calculate beginning and ending for the analyse videos
strings=[]
# for i in range (10,37):
#     string = 'p' + str(i)
#     strings.append(string)
strings = ['']
folders = ['']
start = 0
path = '/content/drive/MyDrive/neuroproj/Prizzi_Mura_norobot' #the file is in our folder
path_to_write='/content/drive/MyDrive/neuroproj/Prizzi_Mura_norobot'


for litstring in strings:
    print(litstring)
    sessionpat = 0

    # mydirectory = greatgreatgrandpath + '\\Resilio Sync\\Dados\\'+litstring+'\\'+listfolder
    # mydirectory = 'C:\\Users\\User\\Desktop\\data_laura\\data_laura\\Dados\\'+litstring+'\\'+listfolder
    # mydirectory = greatgrandppath + '\\Dados\\'+litstring+'\\'+listfolder
    mydirectory = path + litstring
    listafiles = os.listdir(mydirectory) # Extract all the files in a directory

    modelo = Modelo()

    head = HeadBB()
    variabiless=0

    ## Pick the files with different names
    # vecex      = [s for s in listafiles if 'exercise' in s] # Pick the files with the name exercise
    vectempvid = [s for s in listafiles if 'tempos' in s]
    vecvid     = [s for s in listafiles if 'video' in s and not 'yolo' in s and not 'mtcnn' in s and not 'labels' in s and not 'prueba' in s]
    #vecjp      = [s for s in listafiles if 'jointpoints' in s and not 'jointpointschild' in s and not 'eval' in s]
    #vecjpc     = [s for s in listafiles if 'jointpointschild' in s and not 'eval' in s]
    #vecjpo     = [s for s in listafiles if 'jointpointsother' in s and not 'eval' in s]
    veck       = [s for s in listafiles if 'keypoints' in s and not 'keypointschild' in s and not 'eval' in s]
    veckc      = [s for s in listafiles if 'keypointschild' in s and not 'eval' in s]

    print(vecvid)
    # print(vecjp)
    # print(vecjpc)
    # print(veck)
    # print(veckc)
   # if not vectempvid or not vecvid or not vecjp or not vecjpc or not veck or not veckc:
    #    continue

   # joinvecjp = []
    #if len(vecjpo)>0:
     #   for j in vecjp:
     #      for jc in vecjpc:
     #           for jo in vecjpo:
     #              if j.split('jointpoints')[1] == jc.split('jointpointschild')[1]:
     #                 joinvecjp.append([j, jc, jo])
     #                p3 = True
  #  else:
   #     for j in vecjp:
    #        for jc in vecjpc:
     #           if j.split('jointpoints')[1] == jc.split('jointpointschild')[1]:
      #                  joinvecjp.append([j, jc])
       #                 p3 = False

    # joinvecjp = list(map(lambda x, y:[x,y] if x.split('jointpoints')[1] == y.split('jointpointschild')[1] else [None, None], vecjp, vecjpc))
    # print(joinvecjp)
    idvid = 1
    #conjuntosjoints = []
    novalista = []
    ## Correspond the jointpoints to the respective videos

    # for linj in joinvecjp:

        # print('Getting jointpoints timestamp')
        # if linj[0] is None or linj[1] is None:
        #     # conjuntosjoints.append([None, None])
        #     continue
     #   separation = linj[0].split('_')
      #  separation2 = separation[1].split('.')
        # totaldate = '2019'+ separation[0][-4:] + '_' + separation2[0]
        # if listfolder[0:4] == '2020':
        #     totaldate = '2020'+ separation[0][-4:] + '_' + separation2[0]
        # else:
        #     totaldate = '2021'+ separation[0][-4:] + '_' + separation2[0]
        # tempostamp = datetime.datetime.strptime(totaldate + "+0000", "%Y%m%d_%H%M%S%z").timestamp()

        # if idvid < len(vecvid):
        #     current_video = float(vecvid[idvid][5:-4])

        #     if tempostamp > current_video:
        #         conjuntosjoints.append(novalista)
        #         novalista = []
        #         idvid +=1
       # novalista.append(linj)

    #conjuntosjoints.append(novalista)
    #print('conjuntosjoints', conjuntosjoints)

    for idl, l in enumerate(vectempvid): #it reads the time. starting and the end are defined associated to when we have the points.

        if len(vecvid) == 0:
            continue

        # print('Reading video times')

        # Reading times of the frames of the videos

        temposvids = HeadBB.readtimings(mydirectory + '/' + l)



        # Construct the video
        v_reader = imageio.get_reader(mydirectory + '/' + vecvid[idl])
        k = 0


        fps = v_reader.get_meta_data()['fps']
        if idl == 0:
            if writing_mode == 1:
                outfile = path_to_write + '/' + 'video_gaze360.mp4'
                out = imageio.get_writer(outfile,fps=fps)
            f4 = open(path_to_write + '/' + 'angle_gaze360.txt', 'w')

        #analisedlist = conjuntosjoints[0][idl]

        # print(vecvid[idl])

        # import pdb; pdb.set_trace()

        # if analisedlist is None:
        #     continue


        #for idlinj, linj in enumerate(analisedlist):



        # if linj is None:
        #     continue

       # print(linj)

        print('Getting video starting and ending time')
        # Choose which part of the video will be analysed (define begin and end) and extraction of the bounding boxes around the head and eyes

        inicio=0
########################################################################
        # Calcola il numero di frame
        try:
          num_frames = v_reader.count_frames()
        except ValueError:
        # Se il numero di frame non può essere determinato direttamente
          num_frames = sum(1 for _ in v_reader)

        fim=num_frames-1

        # Stampa il numero di frame
        print(f"Numero totale di frame nel video: {num_frames}")
########################################################################

        print('Video cut first part')


        # Go until the point of the video that was defined for the beginning
        print('kbefore',k)
        while(k<inicio): #this part does the real calculation
            v_reader.get_next_data()
            k = k+1
        print('kafter',k)


        print('Starting index: ', inicio)
        print('Ending index: ', fim)
        print('Ending time', temposvids[fim])

        print('Listing video frames')
       # jointpts  = open(mydirectory + '/' + linj[0]).read().splitlines()
        #jointptsc  = open(mydirectory + '/' + linj[1]).read().splitlines()
       # if p3 == True:
        #    jointptso  = open(mydirectory + '/' + linj[2]).read().splitlines()


        v = []
        listk = []
        tracking_id = dict()
        skapa = 0

        WIDTH, HEIGHT =  960, 720 # 1920, 1080

        print('Output video creation\n\n')

        # Predict Gaze in video

        fps = v_reader.get_meta_data()['fps']

        color_encoding = []
        for i in range(1000): color_encoding.append([random.randint(0,254),random.randint(0,254),random.randint(0,254)])

        W = max(int(fps//8),1)
        print('fps',fps)
        print('W',W)


        variabiless = variabiless + 1
        gazes = dict()

        turn = 0
        for kapa in range(inicio,fim+1):
            ## list every frame between inicio(beginning) and fim(ending)
            # print('kapa',kapa)
            image = v_reader.get_next_data()
            k=k+1

            final_results = dict()
            im_width = image.shape[1]
            im_height = image.shape[0]
            # v.append(image)
            im = Image.fromarray(image)
            bboxes, scores, classes = yolo.detect(im) #we give the image to YOLO

            print(temposvids[kapa],kapa/num_frames*100 , '%')

###############################################AGGIUNTO DA NOI

            if len(bboxes) <=1: #modificare per stabilire le condizioni sotto cui salva le bboxes
                continue

            if len(bboxes) > 2: #AGGIUNTO DA NOI
              sorted_indices = np.argsort(scores)[::-1]
              bboxes_sorted= bboxes[sorted_indices]
              scores_sorted = scores[sorted_indices]
              classes = classes[sorted_indices]
              bboxes=[]
              scores=[]
              bboxes=bboxes_sorted[0:2]
              scores=scores_sorted[0:2]

#####################################################

            # if np.isnan(center).any() or np.isinf(center).any() or np.isnan(center_c).any() or np.isinf(center_c).any():
            #     continue

            nbox = 0
            for bbox in bboxes:
                nbox=nbox+1
                orderedbb = [bbox[1], bbox[0], bbox[3], bbox[2]]
                eyes= [(orderedbb[0]+orderedbb[2])/2.0, (0.65*orderedbb[1]+0.35*orderedbb[3])]
                final_results[nbox] = (orderedbb,eyes)



            # tracking_id[kapa-inicio] = final_results
            v.append(image)
            listk.append(k-1)
            # print('video_length', len(v))
            tracking_id[skapa] = final_results
            skapa = skapa + 1

            if skapa >= 7*W:
                i = skapa-7*W
                while i < 3*W + 1:
                    if turn > 0:
                        i = 3*W
                    # print('iframes: ',i)
                    # print('current_i', i)
                    im = v[i]
                    image = im.copy()
                    image = cv2.resize(image,(WIDTH,HEIGHT))
                    image = image.astype(float)
                    if i in tracking_id:
                        for id_t in tracking_id[i].keys():
                            input_image = torch.zeros(7,3,224,224)
                            count = 0
                            # print('jmin', i-3*W)
                            # print('jmax', i+4*W)
                            for j in range(i-3*W,i+4*W,W):
                                # import pdb; pdb.set_trace()
                                if j in tracking_id and id_t in tracking_id[j]:
                                    new_im = Image.fromarray(v[j],'RGB')
                                    bbox,eyes = tracking_id[j][id_t]

                                else:
                                    new_im = Image.fromarray(v[i],'RGB')
                                    bbox,eyes = tracking_id[i][id_t]

                                new_im = new_im.crop((bbox[0],bbox[1],bbox[2],bbox[3]))
                                input_image[count,:,:,:] = image_normalize(transforms.ToTensor()(transforms.Resize((224,224))(new_im)))
                                count = count+1


                            bbox,eyes = tracking_id[i][id_t]
                            bbox = np.asarray(bbox).astype(int)

                            output_gaze,confidence = model(input_image.view(1,7,3,224,224).cuda())
                            gaze = spherical2cartesial(output_gaze).detach().numpy()
                            eyes = np.asarray(eyes).astype(float)
                            eyes[0],eyes[1] = WIDTH*eyes[0]/float(v[i].shape[1]),HEIGHT*eyes[1]/float(v[i].shape[0])

                            gaze = gaze.reshape((-1))

                            # print('Gaze :', gaze, '  Frame : ', i)
                            # print('Eyes :', eyes)

                            # import pdb; pdb.set_trace()

                            m = ( (eyes[1]-gaze[1]*100) - eyes[1] ) / ( (eyes[0]-gaze[0]*100) - eyes[0] )

                            q = -eyes[0]*m + eyes[1]


                            image = image.astype(np.uint8)
                            bbox[0],bbox[2] = WIDTH*bbox[0]/v[i].shape[1],WIDTH*bbox[2]/v[i].shape[1]
                            bbox[1],bbox[3] = HEIGHT*bbox[1]/v[i].shape[0],HEIGHT*bbox[3]/v[i].shape[0]
                            image = cv2.rectangle(image, (bbox[0],bbox[1]), (bbox[2],bbox[3]),color_encoding[min(id_t,900)])
                            image = cv2.arrowedLine(image, (int(eyes[0]),int(eyes[1])), (int(eyes[0]-gaze[0]*100),int(eyes[1]-gaze[1]*100)),color_encoding[min(id_t,900)], thickness = 6)
                            image = image.astype(float)

                            confidencef, output_gazef = tensor2np(confidence,output_gaze)
                            # print('current frame', i+inicio+turn)
                            if len(tracking_id[i].keys())>=1:
                                f4.write(str(listk[i]-start) + " " + str(id_t) + " " + str(confidencef[0]) + " " + str(confidencef[1]) + " " + str(output_gazef[0]) + " " + str(output_gazef[1])+ " " + str(bbox[0])+ " " + str(bbox[1])+ " " + str(bbox[2])+ " " + str(bbox[3]) +" "+str(eyes[0])+ " "+ str(eyes[1])+"\n")
                                print('numero di bb salvate:' , len(bboxes))
                                print('Confidenza:' , scores)


                    image = image.astype(np.uint8)
                    if i in tracking_id and len(tracking_id[i].keys()) >= 1:
                        if writing_mode == 1:
                            out.append_data(image)
                    i = i+1

                skapa = skapa-1
                # print('skapa_after',skapa)
                newv = v[1:].copy()
                v= []
                v= newv
                newlistk = listk[1:].copy()
                listk= []
                listk= newlistk
                new_tracking_id = tracking_id.copy()
                tracking_id = dict()
                turn = turn + 1
                for key, value in new_tracking_id.items():
                    if key-1 >=0:
                        tracking_id[key-1] = new_tracking_id[key]
    if writing_mode == 1:
        out.close()
        # writing_mode = 0
    f4.close()
        # k = modelo.listvideo(inicio, fim, v_reader, k, mydirectory, temposvids, fps, newtimingslist, tracking_id)
        # print('kafterlist',k)





['video1732612687.8965955.avi']
Getting video starting and ending time
Numero totale di frame nel video: 858
Video cut first part
kbefore 0
kafter 0
Starting index:  0
Ending index:  857
Ending time 1732612778.402952
Listing video frames
Output video creation


fps 30.0
W 3
1732612688.843959 0.0 %
1732612688.9525564 0.11655011655011654 %
1732612689.0470405 0.2331002331002331 %
1732612689.1456006 0.34965034965034963 %
1732612689.2359579 0.4662004662004662 %
1732612689.3247263 0.5827505827505828 %
1732612689.41907 0.6993006993006993 %
1732612689.5093873 0.8158508158508158 %
1732612689.6019626 0.9324009324009324 %
1732612689.6954174 1.048951048951049 %
1732612689.782984 1.1655011655011656 %
1732612689.875267 1.282051282051282 %
1732612689.977744 1.3986013986013985 %
1732612690.0800781 1.5151515151515151 %
1732612690.1716833 1.6317016317016315 %
1732612690.2642043 1.7482517482517483 %
1732612690.3542051 1.8648018648018647 %
1732612690.45039 1.9813519813519813 %
1732612690.5427406 2.097902

In [1]:

## Define model
writing_mode = 1
## Calculate beginning and ending for the analyse videos
strings=[]
# for i in range (10,37):
#     string = 'p' + str(i)
#     strings.append(string)
strings = ['']
folders = ['']
start = 0
path = "/content/drive/MyDrive/neuroproj/Prizzi_Mura_robot" #the file is in our folder
path_to_write= '/content/drive/MyDrive/neuroproj/Prizzi_Mura_robot'


for litstring in strings:
    print(litstring)
    sessionpat = 0

    # mydirectory = greatgreatgrandpath + '\\Resilio Sync\\Dados\\'+litstring+'\\'+listfolder
    # mydirectory = 'C:\\Users\\User\\Desktop\\data_laura\\data_laura\\Dados\\'+litstring+'\\'+listfolder
    # mydirectory = greatgrandppath + '\\Dados\\'+litstring+'\\'+listfolder
    mydirectory = path + litstring
    listafiles = os.listdir(mydirectory) # Extract all the files in a directory

    modelo = Modelo()

    head = HeadBB()
    variabiless=0

    ## Pick the files with different names
    # vecex      = [s for s in listafiles if 'exercise' in s] # Pick the files with the name exercise
    vectempvid = [s for s in listafiles if 'tempos' in s]
    vecvid     = [s for s in listafiles if 'video' in s and not 'yolo' in s and not 'mtcnn' in s and not 'labels' in s and not 'prueba' in s]
    #vecjp      = [s for s in listafiles if 'jointpoints' in s and not 'jointpointschild' in s and not 'eval' in s]
    #vecjpc     = [s for s in listafiles if 'jointpointschild' in s and not 'eval' in s]
    #vecjpo     = [s for s in listafiles if 'jointpointsother' in s and not 'eval' in s]
    veck       = [s for s in listafiles if 'keypoints' in s and not 'keypointschild' in s and not 'eval' in s]
    veckc      = [s for s in listafiles if 'keypointschild' in s and not 'eval' in s]

    print(vecvid)
    # print(vecjp)
    # print(vecjpc)
    # print(veck)
    # print(veckc)
   # if not vectempvid or not vecvid or not vecjp or not vecjpc or not veck or not veckc:
    #    continue

   # joinvecjp = []
    #if len(vecjpo)>0:
     #   for j in vecjp:
     #      for jc in vecjpc:
     #           for jo in vecjpo:
     #              if j.split('jointpoints')[1] == jc.split('jointpointschild')[1]:
     #                 joinvecjp.append([j, jc, jo])
     #                p3 = True
  #  else:
   #     for j in vecjp:
    #        for jc in vecjpc:
     #           if j.split('jointpoints')[1] == jc.split('jointpointschild')[1]:
      #                  joinvecjp.append([j, jc])
       #                 p3 = False

    # joinvecjp = list(map(lambda x, y:[x,y] if x.split('jointpoints')[1] == y.split('jointpointschild')[1] else [None, None], vecjp, vecjpc))
    # print(joinvecjp)
    idvid = 1
    #conjuntosjoints = []
    novalista = []
    ## Correspond the jointpoints to the respective videos

    # for linj in joinvecjp:

        # print('Getting jointpoints timestamp')
        # if linj[0] is None or linj[1] is None:
        #     # conjuntosjoints.append([None, None])
        #     continue
     #   separation = linj[0].split('_')
      #  separation2 = separation[1].split('.')
        # totaldate = '2019'+ separation[0][-4:] + '_' + separation2[0]
        # if listfolder[0:4] == '2020':
        #     totaldate = '2020'+ separation[0][-4:] + '_' + separation2[0]
        # else:
        #     totaldate = '2021'+ separation[0][-4:] + '_' + separation2[0]
        # tempostamp = datetime.datetime.strptime(totaldate + "+0000", "%Y%m%d_%H%M%S%z").timestamp()

        # if idvid < len(vecvid):
        #     current_video = float(vecvid[idvid][5:-4])

        #     if tempostamp > current_video:
        #         conjuntosjoints.append(novalista)
        #         novalista = []
        #         idvid +=1
       # novalista.append(linj)

    #conjuntosjoints.append(novalista)
    #print('conjuntosjoints', conjuntosjoints)

    for idl, l in enumerate(vectempvid): #it reads the time. starting and the end are defined associated to when we have the points.

        if len(vecvid) == 0:
            continue

        # print('Reading video times')

        # Reading times of the frames of the videos

        temposvids = HeadBB.readtimings(mydirectory + '/' + l)



        # Construct the video
        v_reader = imageio.get_reader(mydirectory + '/' + vecvid[idl])
        k = 0


        fps = v_reader.get_meta_data()['fps']
        if idl == 0:
            if writing_mode == 1:
                outfile = path_to_write + '/' + 'video_gaze360.mp4'
                out = imageio.get_writer(outfile,fps=fps)
            f4 = open(path_to_write + '/' + 'angle_gaze360.txt', 'w')

        #analisedlist = conjuntosjoints[0][idl]

        # print(vecvid[idl])

        # import pdb; pdb.set_trace()

        # if analisedlist is None:
        #     continue


        #for idlinj, linj in enumerate(analisedlist):



        # if linj is None:
        #     continue

       # print(linj)

        print('Getting video starting and ending time')
        # Choose which part of the video will be analysed (define begin and end) and extraction of the bounding boxes around the head and eyes

        inicio=0
########################################################################
        # Calcola il numero di frame
        try:
          num_frames = v_reader.count_frames()
        except ValueError:
        # Se il numero di frame non può essere determinato direttamente
          num_frames = sum(1 for _ in v_reader)

        fim=num_frames-1

        # Stampa il numero di frame
        print(f"Numero totale di frame nel video: {num_frames}")
########################################################################

        print('Video cut first part')


        # Go until the point of the video that was defined for the beginning
        print('kbefore',k)
        while(k<inicio): #this part does the real calculation
            v_reader.get_next_data()
            k = k+1
        print('kafter',k)


        print('Starting index: ', inicio)
        print('Ending index: ', fim)
        print('Ending time', temposvids[fim])

        print('Listing video frames')
       # jointpts  = open(mydirectory + '/' + linj[0]).read().splitlines()
        #jointptsc  = open(mydirectory + '/' + linj[1]).read().splitlines()
       # if p3 == True:
        #    jointptso  = open(mydirectory + '/' + linj[2]).read().splitlines()


        v = []
        listk = []
        tracking_id = dict()
        skapa = 0

        WIDTH, HEIGHT =  960, 720 # 1920, 1080

        print('Output video creation\n\n')

        # Predict Gaze in video

        fps = v_reader.get_meta_data()['fps']

        color_encoding = []
        for i in range(1000): color_encoding.append([random.randint(0,254),random.randint(0,254),random.randint(0,254)])

        W = max(int(fps//8),1)
        print('fps',fps)
        print('W',W)


        variabiless = variabiless + 1
        gazes = dict()

        turn = 0
        for kapa in range(inicio,fim+1):
            ## list every frame between inicio(beginning) and fim(ending)
            # print('kapa',kapa)
            print(temposvids[kapa],kapa/num_frames*100 , '%')
            image = v_reader.get_next_data()
            k=k+1

            final_results = dict()
            im_width = image.shape[1]
            im_height = image.shape[0]
            # v.append(image)
            im = Image.fromarray(image)
            bboxes, scores, classes = yolo.detect(im) #we give the image to YOLO

            print(temposvids[kapa])

###############################################AGGIUNTO DA NOI

            if len(bboxes) <=1: #modificare per stabilire le condizioni sotto cui salva le bboxes
                continue

            if len(bboxes) > 2: #AGGIUNTO DA NOI
              sorted_indices = np.argsort(scores)[::-1]
              bboxes_sorted= bboxes[sorted_indices]
              scores_sorted = scores[sorted_indices]
              classes = classes[sorted_indices]
              bboxes=[]
              scores=[]
              bboxes=bboxes_sorted[0:2]
              scores=scores_sorted[0:2]

#####################################################

            # if np.isnan(center).any() or np.isinf(center).any() or np.isnan(center_c).any() or np.isinf(center_c).any():
            #     continue

            nbox = 0
            for bbox in bboxes:
                nbox=nbox+1
                orderedbb = [bbox[1], bbox[0], bbox[3], bbox[2]]
                eyes= [(orderedbb[0]+orderedbb[2])/2.0, (0.65*orderedbb[1]+0.35*orderedbb[3])]
                final_results[nbox] = (orderedbb,eyes)



            # tracking_id[kapa-inicio] = final_results
            v.append(image)
            listk.append(k-1)
            # print('video_length', len(v))
            tracking_id[skapa] = final_results
            skapa = skapa + 1

            if skapa >= 7*W:
                i = skapa-7*W
                while i < 3*W + 1:
                    if turn > 0:
                        i = 3*W
                    # print('iframes: ',i)
                    # print('current_i', i)
                    im = v[i]
                    image = im.copy()
                    image = cv2.resize(image,(WIDTH,HEIGHT))
                    image = image.astype(float)
                    if i in tracking_id:
                        for id_t in tracking_id[i].keys():
                            input_image = torch.zeros(7,3,224,224)
                            count = 0
                            # print('jmin', i-3*W)
                            # print('jmax', i+4*W)
                            for j in range(i-3*W,i+4*W,W):
                                # import pdb; pdb.set_trace()
                                if j in tracking_id and id_t in tracking_id[j]:
                                    new_im = Image.fromarray(v[j],'RGB')
                                    bbox,eyes = tracking_id[j][id_t]

                                else:
                                    new_im = Image.fromarray(v[i],'RGB')
                                    bbox,eyes = tracking_id[i][id_t]

                                new_im = new_im.crop((bbox[0],bbox[1],bbox[2],bbox[3]))
                                input_image[count,:,:,:] = image_normalize(transforms.ToTensor()(transforms.Resize((224,224))(new_im)))
                                count = count+1


                            bbox,eyes = tracking_id[i][id_t]
                            bbox = np.asarray(bbox).astype(int)

                            # output_gaze,confidence = model(input_image.view(1,7,3,224,224).cuda())
                            output_gaze, confidence = model(input_image.view(1, 7, 3, 224, 224))

                            gaze = spherical2cartesial(output_gaze).detach().numpy()
                            eyes = np.asarray(eyes).astype(float)
                            eyes[0],eyes[1] = WIDTH*eyes[0]/float(v[i].shape[1]),HEIGHT*eyes[1]/float(v[i].shape[0])

                            gaze = gaze.reshape((-1))

                            # print('Gaze :', gaze, '  Frame : ', i)
                            # print('Eyes :', eyes)

                            # import pdb; pdb.set_trace()

                            m = ( (eyes[1]-gaze[1]*100) - eyes[1] ) / ( (eyes[0]-gaze[0]*100) - eyes[0] )

                            q = -eyes[0]*m + eyes[1]


                            image = image.astype(np.uint8)
                            bbox[0],bbox[2] = WIDTH*bbox[0]/v[i].shape[1],WIDTH*bbox[2]/v[i].shape[1]
                            bbox[1],bbox[3] = HEIGHT*bbox[1]/v[i].shape[0],HEIGHT*bbox[3]/v[i].shape[0]
                            image = cv2.rectangle(image, (bbox[0],bbox[1]), (bbox[2],bbox[3]),color_encoding[min(id_t,900)])
                            image = cv2.arrowedLine(image, (int(eyes[0]),int(eyes[1])), (int(eyes[0]-gaze[0]*100),int(eyes[1]-gaze[1]*100)),color_encoding[min(id_t,900)], thickness = 6)
                            image = image.astype(float)

                            confidencef, output_gazef = tensor2np(confidence,output_gaze)
                            # print('current frame', i+inicio+turn)
                            if len(tracking_id[i].keys())>=1:
                                f4.write(str(listk[i]-start) + " " + str(id_t) + " " + str(confidencef[0]) + " " + str(confidencef[1]) + " " + str(output_gazef[0]) + " " + str(output_gazef[1])+ " " + str(bbox[0])+ " " + str(bbox[1])+ " " + str(bbox[2])+ " " + str(bbox[3]) +" "+str(eyes[0])+ " "+ str(eyes[1])+"\n")
                                print('numero di bb salvate:' , len(bboxes))
                                print('Confidenza:' , scores)


                    image = image.astype(np.uint8)
                    if i in tracking_id and len(tracking_id[i].keys()) >= 1:
                        if writing_mode == 1:
                            out.append_data(image)
                    i = i+1

                skapa = skapa-1
                # print('skapa_after',skapa)
                newv = v[1:].copy()
                v= []
                v= newv
                newlistk = listk[1:].copy()
                listk= []
                listk= newlistk
                new_tracking_id = tracking_id.copy()
                tracking_id = dict()
                turn = turn + 1
                for key, value in new_tracking_id.items():
                    if key-1 >=0:
                        tracking_id[key-1] = new_tracking_id[key]
    if writing_mode == 1:
        out.close()
        # writing_mode = 0
    f4.close()
        # k = modelo.listvideo(inicio, fim, v_reader, k, mydirectory, temposvids, fps, newtimingslist, tracking_id)
        # print('kafterlist',k)




NameError: name 'os' is not defined